In [ ]:
#%load_ext nb_black
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten
#from tensorflow.python.framework.ops import disable_eager_execution

# from keras.layers.normalization import BatchNormalization

# from keras.layers.convolutional import Conv2D
from tensorflow.keras.optimizers import Adam
#from tensorflow.keras.callbacks import TensorBoard
from collections import deque
from tqdm import tqdm
import time
import numpy as np
import pandas as pd
import random
import timeit
from datetime import datetime
from sklearn.preprocessing import MinMaxScaler
import pickle
import os
#os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
#%load_ext tensorboard
#import dask.dataframe as dd

#disable_eager_execution()

In [ ]:
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    try:
        # Restrict TensorFlow to only use the fourth GPU
        tf.config.experimental.set_visible_devices(gpus[0], 'GPU')

        # Currently, memory growth needs to be the same across GPUs
        for gpu in gpus:
            tf.config.experimental.set_memory_growth(gpu, True)
        logical_gpus = tf.config.experimental.list_logical_devices('GPU')
        print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
    except RuntimeError as e:
        # Memory growth must be set before GPUs have been initialized
        print(e)

In [ ]:
tf.config.experimental.list_physical_devices()

In [ ]:
train_filename = "data\EURUSD_2019Day_HALF_UTCplus2_prepro.csv"
test_filename = "data\\EURUSD_2019Day_HALF_UTCplus2_prepro.csv"
#filename = "data\\testblok.csv"


# dataframe = pd.read_csv('data\EURUSD_2017_HALF_UTCplus2_prepro.csv')

In [ ]:
SYMBOL = "EURUSD"
SYMBOL_FACTOR = 10_000
BOOK_NAME = "orderbook"
LOAD_SCALER = True
EPOCHS = 3
REPLAY_MEMORY_SIZE = 1000_000
MIN_REPLAY_MEMORY_SIZE = 100_000
MODEL_NAME = "LSTM3_DQN_"
BATCHSIZE = 4096
MINIBATCH_SIZE = 128
UPDATE_TARGET_EVERY = 5
ENVIROMENT_OBSERVATION_SPACE = 139  # 138 from csv/signals + "pip saldo"
ACTION_SPACE = (
    4  # The four actions available for the Agent (Nothing, Buy, Sell, Close open order)
)
LEARNING_RATE = 0.1

DISCOUNT = 0.90  # gamme
EPISODES = 24
EPSILON_DECAY = 0.95
MIN_EPSILON = 0.01
END_EPSILON_DECAYING = EPISODES // 2
REPLAY = []
LEARNING_PROGRESS = []
MIN_WIN_LOSS = 10
epsilon = 1

difference = 0
total_win_loss = 0
prev_win_loss = 0
prev_hour = 0
orders = ""
#log_dir = f"logs\\{MODEL_NAME}_{int(time.time())}"
ep_rewards = []
book_time = int(time.time())
close_market = False


# TENSORBOARD STUFF
#tensorboard = tf.keras.callbacks.TensorBoard(log_dir = f"logs\\{MODEL_NAME}_{int(time.time())}")

log_dir="logs\\" + datetime.now().strftime("%Y%m%d_%H%M%S")
tensorboard = tf.keras.callbacks.TensorBoard(log_dir=log_dir, histogram_freq=1)
file_writer = tf.summary.create_file_writer(log_dir + "\metrics")
#file_writer.set_as_default()

#, update_freq= EPISODES, histogram_freq=1
#lr_schedule_callback = LearningRateScheduler(schedule)



# Create models folder
if not os.path.isdir("models"):
    os.makedirs("models")

# For more repetitive results
random.seed(1)
np.random.seed(1)
tf.random.set_seed(1)

In [ ]:
def preprocess(filename):
    global xdata, scaler
    # Set initial points(pips) earned to 0
    points_init = 0
    points = np.array([points_init]).astype("float64")
    point_resp = np.reshape(points, (-1, 1))
    data = pd.read_csv(filename)
    
    np_df = data.to_numpy()
    win_loss_proxy = np.random.uniform(
        low=-1.0, high=1.0, size=data.shape[0]
    ).reshape(-1, 1)
    proxy_array = np.concatenate((np_df, win_loss_proxy), axis=1)
    scaler = MinMaxScaler(feature_range=(-1, 1))
    scl = scaler.fit(proxy_array)


    with open(f"scaler/{SYMBOL}_scaler_{int(time.time())}.pkl", "wb") as f:
        pickle.dump(scaler, f)

    xdata = proxy_array[:, :-1]
    first_state = xdata[0:1, :]
    current_state = np.concatenate((first_state, point_resp), axis=1)
    return current_state.flatten(), xdata
   

In [ ]:
def to_datetime(current_state):
    current_flat = current_state.flatten()
    datetime_object = datetime(
        year=int(current_flat[0]),
        month=int(current_flat[1]),
        day=int(current_flat[2]),
        hour=int(current_flat[3]),
        minute=int(current_flat[4]),
    )
    # print(datetime_object)
    return datetime_object

In [ ]:
def action_check(current_state, action, orders, close_market):
    # Check action if its legal and then send it to market or return a legal action if chosen action is illegal.
    # 0 = do nothing   #1 = buy   #2 = sell   #3 = close order
    if close_market:
        if orders == "long":  # Is it end of day
            action = 3
        elif orders == "short":
            action = 3
        else:
            action = 0
    else:
        if action == 0:
            pass

        elif action == 1:
            if orders == "long":
                action = 0
            elif orders == "short":
                action = 3

        elif action == 2:
            if orders == "long":
                action = 3
            elif orders == "short":
                action = 0

        elif action == 3:
            if orders == "long":
                action = 3
            elif orders == "short":
                action = 3
            else:
                action = 0

    return action

In [ ]:
def book_keeping(orders, orderprice, close_price, win_loss, current_state):
    global total_win_loss
    timestamp = to_datetime(current_state)
    time_string = timestamp.strftime("%Y.%m.%d, %H:%M:%S")
    total_win_loss += win_loss
    
    # Create a numpy array with all the trade/state info for the current state
    print(time_string + ", " + orders + ", " + str(win_loss) + ", " + str(total_win_loss))
    trade_info = (np.array([time_string, orders, orderprice, close_price, win_loss, total_win_loss]).reshape([1, -1]).flatten())
    
    state_sub = current_state.flatten()[5:138]
    state_info = np.concatenate((trade_info, state_sub)).flatten()

    with open(f"output/{BOOK_NAME}_{book_time}.csv", "a") as f:
        np.savetxt(f, state_info, fmt="%s", newline=" ", delimiter=",")
        f.write("\n")

    return total_win_loss

In [ ]:
def to_market(current_state, action, timestep, win_loss):
    global orders, order_price, terminal_state, total_win_loss

    current = current_state.flatten()
    if action == 0:
        if orders == 'long':
            win_loss = current[5] - order_price
        
        elif orders == 'short':
            win_loss = order_price - current[6]
        
        else:
            pass
        close_order = False
        #tot_w_l = 0
    
    if action == 1:
        #Send buy order to market
        orders = 'long'
        order_price =  current[6] #Get 'ask' price from current state and saves it
        close_order = False
        #tot_w_l = 0
    
    elif action == 2:
        #Send sell order to market
        orders = 'short'
        order_price = current[5] #Get 'bid' price from current state and saves it
        close_order = False
        #tot_w_l = 0
    
    elif action == 3 and orders =='long':
        #Send close order to market
        close_price = current[5]
        win_loss = (close_price - order_price)*SYMBOL_FACTOR #close long order
        total_win_loss = book_keeping(orders, order_price, close_price, win_loss, current_state) #Send trade info to CSV file
        close_order = True
        orders =''
    
    elif action == 3 and orders =='short':
        #Send close order to market
        close_price = current[6]
        win_loss = (order_price - close_price)*SYMBOL_FACTOR #close short order
        total_win_loss = book_keeping(orders, order_price, close_price, win_loss, current_state) #Send trade info to CSV file
        close_order = True
        orders =''
    
    #Update timestep and state
    timestep += 1
    if timestep == len(xdata-1):
        terminal_state = True
    else:
        terminal_state = False
    update_state = xdata[timestep]
    update_state2 = update_state.flatten()
    if win_loss > 100:
        win_loss_adj = 100
    elif win_loss < -100:
        win_loss_adj = -100
    else:
        win_loss_adj = win_loss
    wl = np.array(win_loss_adj/100).astype('float64')
    wl_scaled = wl.flatten()
    new_state = (np.concatenate((update_state2,wl_scaled))).reshape(-1,139)
    
    return new_state, terminal_state, win_loss, close_order, total_win_loss, timestep, orders


In [ ]:
def get_reward(win_loss, prev_win_loss, close):
    difference = win_loss - prev_win_loss
    close_bonus = 0
    
    #BONUS FOR CLOSED ORDERS
    if close:
        if win_loss < 0:
            close_bonus = -0.05
        elif win_loss == 0:
            close_bonus = 0
        elif win_loss > 0:
            close_bonus = 0.05
    
    #NEGATIVE
    if win_loss < -10:  # Loss - minus
        step_bonus = -0.005
    elif win_loss > -10 and win_loss < 0:  # Loss - minus
        step_bonus = -0.002
    
    #ZERO    
    elif win_loss == 0:  # Loss - minus#Loss
        step_bonus = 0
        
    #POSITIVE
    elif win_loss > 0 and win_loss < 10:  # win
        step_bonus = 0.002
    elif win_loss > 10 and win_loss < 20:  # win
        step_bonus = 0.005
    elif win_loss > 20:  # win
        step_bonus = 0.008
    
    reward = difference * step_bonus + close_bonus

    return reward

In [ ]:
def end_day(current_state):
    sample_time = to_datetime(current_state)
    #if int(sample_time.hour) == 0 and int(sample_time.minute) > 5:
    if int(sample_time.hour) == 23 and int(sample_time.minute) > 57:
        day_over = True
    else:
        day_over = False
    return day_over

In [ ]:
def episode_done(current_state):
    global prev_hour
    current_flat = current_state.flatten()
    if (current_flat[3] != prev_hour): 
        Episode_done = True
    else:
        Episode_done = False
    prev_hour = current_flat[3]
    return Episode_done

#### Model Class

In [ ]:
class DQNAgent:
    def __init__(self):
        
        #main model # gets trained every step
        self.model = self.create_model()
        
        #Target model this is what we .predict against every step
        self.target_model = self.create_model()
        self.target_model.set_weights(self.model.get_weights())
        
        self.replay_memory = deque(maxlen=REPLAY_MEMORY_SIZE)
        
        self.tensorboard = tensorboard #TensorBoard(log_dir=f"logs/{MODEL_NAME}_{int(time.time())}")
        self.target_update_counter = 0
        
    def create_model(self):
        model = Sequential()
        model.add(Dense(128, input_shape=(139,), activation='relu'))
        model.add(Dropout(0.2))
        
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        
        model.add(Dense(128, activation='relu'))
        model.add(Dropout(0.2))
        model.add(Dense(4, activation='softmax'))
    
        #Model compile settings:
        opt = tf.keras.optimizers.Adam(lr=0.001, decay=1e-6)
        
        # Compile model
        model.compile(
        loss='categorical_crossentropy',
        optimizer=opt,
        metrics=['accuracy']
        )
        print(model.summary())
        return model
    
    def update_replay_memory(self, transition):
        self.replay_memory.append(transition)
    
    def train(self, terminal_state):
        global export
        if len(self.replay_memory) < MIN_REPLAY_MEMORY_SIZE:
            return
        
        minibatch = random.sample(self.replay_memory, MINIBATCH_SIZE)
        
        current_states = np.array([transition[0] for transition in minibatch])
        #print('0_',current_states.shape)
        current_qs_list = self.model.predict(current_states)
        #print('1_',current_qs_list)
    
        new_states = np.array([transition[3] for transition in minibatch])
        
        future_qs_list = self.target_model.predict(new_states)

        X = []
        y = []
        
        # Now we need to enumerate our batches
        for index, (current_state, action, reward, new_state, done) in enumerate(minibatch):

            # If not a terminal state, get new q from future states, otherwise set it to 0
            # almost like with Q Learning, but we use just part of equation here
            if not done:
                max_future_q = np.max(future_qs_list[index])
                new_q = reward + DISCOUNT * max_future_q
            else:
                new_q = reward

            # Update Q value for given state
            qs = (current_qs_list[index]).flatten()
            
            #print('22_',qs, qs.shape, [index])
            qs[action] = new_q

            # And append to our training data
            X.append((current_state).reshape(-1))
            y.append(qs)
            #print('X shape: ',np.array(X).shape,' y shape: ',np.array(y).shape)
            
        # Fit on all samples as one batch, log only on terminal state
        self.model.fit(np.array(X), np.array(y), batch_size=MINIBATCH_SIZE, verbose=0, shuffle=False, callbacks=[tensorboard])

        # Update target network counter every episode
        if done:
            self.target_update_counter += 1

        # If counter reaches set value, update target network with weights of main network
        if self.target_update_counter > UPDATE_TARGET_EVERY:
            self.target_model.set_weights(self.model.get_weights())
            self.target_update_counter = 0
            
    # Queries main network for Q values given current observation space (environment state)
    def get_qs(self, state):
        return self.model.predict(scaler.transform(state))
    
agent = DQNAgent()



In [ ]:
for i in range(EPOCHS):
    print("EPOCH #", i, " starting, of ", EPOCHS, "epochs. Time: ")
    if i == EPOCHS - 1:  # the last epoch, use test data set
        EPISODES = 120
        current_state, xdata = preprocess(test_filename)
    else:
        current_state, xdata = preprocess(train_filename)
    win_loss = 0
    step = 1
    # Iterate over episodes
    for episode in tqdm(range(1, EPISODES + 1), ascii=True, unit="episodes"):

        # Update tensorboard step every episode
        agent.tensorboard.step = episode

        # Restarting episode - reset episode reward and step number
        episode_reward = 0

        # Reset flag and start iterating until episode ends
        done = False
        
        while close_market:
            step +=1
            minute = xdata[step:(step+1),4:5]
            if 15 <= minute < 57:
                close_market = False
                done = False
        
        while not done:
            done = episode_done(current_state)
            if end_day(current_state):
                close_market = True
                done = True
                

            # This part stays mostly the same, the change is to query a model for Q values
            if np.random.random() > epsilon:
                # Get action from Q table
                action = np.argmax(agent.get_qs(current_state))
                print("Q-value action",action)
                action = action_check(current_state, action, orders, close_market)
            else:
                # Get random action
                # print("Random action")
                action = np.random.randint(0, 4)
                action = action_check(current_state, action, orders, close_market)

            (
                new_state,
                terminal_state,
                win_loss,
                close,
                total_win_loss,
                step,
                orders,
            ) = to_market(current_state, action, step, win_loss)

            reward = get_reward(win_loss, prev_win_loss, close)
            episode_reward += reward

            # Every step we update replay memory and train main network
            scaled_current = (scaler.transform(current_state.reshape(1, -1))).flatten()
            
            scaled_new_state = (scaler.transform(new_state.reshape(1, -1))).flatten()
            agent.update_replay_memory((scaled_current, action, reward, scaled_new_state, done))
            
            agent.train(done)

            # step += 1

            current_state = new_state

            prev_win_loss = win_loss

            market_close = False

        # Append episode reward to a list and log stats (every given number of episodes)
        ep_rewards.append(episode_reward)

        with file_writer.as_default():
            tf.summary.scalar("Reward", episode_reward, step=episode)
            tf.summary.scalar("Pips", win_loss, step=episode)

        average_reward = sum(ep_rewards) / len(ep_rewards)
        min_reward = min(ep_rewards)
        max_reward = max(ep_rewards)
        #agent.tensorboard.update_stats(
        #    reward_avg=average_reward,
        #    reward_min=min_reward,
        #    reward_max=max_reward,
        #    epsilon=epsilon,
        #)

        # Save model, but only when min reward is greater or equal a set value
        if episode/50 == 0:  #total_win_loss >= MIN_WIN_LOSS:
            agent.model.save(f"models\\{MODEL_NAME}__{SYMBOL}__{int(time.time())}.model")

        # Decay epsilon
        if epsilon > MIN_EPSILON:
            epsilon *= EPSILON_DECAY
            epsilon = max(MIN_EPSILON, epsilon)
    print("EPOCH #", i, " done, of ", epochs, "epochs")

In [ ]:
#agent.model.save(f"models\\{MODEL_NAME}__{SYMBOL}__{int(time.time())}.model")